In [3]:
import numpy as np
import math

In [6]:
def simplified_SMO(C, tolerance, max_iterations, train):
    x, y = split_frame(train)
    train_length = len(x)
    j_tol = 0.000005
    #initialize Lagrange multiplies
    a = np.zeros(train_length)
    b = 0
    number_of_iterations = 0
    while number_of_iterations < max_iterations:
        a_changes = 0
        #iterate over length of training set
        for i in range(0, train_length):
            #calculate the error with the current example
            error_i = calculate_error(x[i], y[i])
            if ((y[i] * error_i) < tolerance * -1 and a[i] < C) or ((y[i] * error_i) > tolerance and a[i] > 0):
                j = get_j(train_length, i)
                a_j_old = a[j]
                a_i_old = a[i]
                L, H = calculate_constraints(C, a[i], a[j], y[i], y[j])
                if L==H:
                    continue
                eta = calculate_eta(x[i],x[j])
                if eta >= 0:
                    continue
                error_j = calcualte_error(x[j], y[j])
                a[j] = calculate_aj(a[j], y[j], error_i, error_j, eta)
                #if j change is miniscule do not need to update i
                if abs(a[j] - a_j_old) < j_tol:
                    continue
                a[i] = calculate_ai(a[i], y[i], y[j], a_j_old, a_j)
                if (a[i] > 0) and (a[i] < C):
                    b = calculate_b1(b, error_i, y[i], a[i], a_i_old, x[i], y[j], a[j], a_j_old, x[j])
                elif (a[j] > 0) and (a[j] < C):
                    b = calculate_b2(b, error_j, y[i], a[i], a_i_old, x[i], x[j], y[j], a[j], a_j_old)
                else:
                    b1 = calculate_b1(b, error_i, y[i], a[i], a_i_old, x[i], y[j], a[j], a_j_old, x[j])
                    b2 = calculate_b2(b, error_j, y[i], a[i], a_i_old, x[i], x[j], y[j], a[j], a_j_old)
                    b = (b1 + b2) / 2
                a_changes+=1
        #if convergence (i.e. no changes to multipliers throughout entire iteration)
        if a_changes == 0:
            number_of_iterations+=1
        else:
            number_of_iterations=0     
                    